# <center>Deepseek企业级Agent项目开发实战</center>

## <center>Part 2. Ollama REST API - api/generate 接口详解 </center>

&emsp;&emsp;Ollama 服务启动后会提供一系列原生 ` REST API` 端点。通过这些`Endpoints`可以在代码环境下与`ollama`启动的大模型进行交互、管理模型和获取相关信息。其中两个`endpoint` 是最重要的，分别是：
  - <font color="red">**POST /api/generate**</font>
  - <font color="red">**POST /api/chat**</font>

&emsp;&emsp;其他端点情况：
  - POST /api/create   
  - POST /api/tags
  - POST /api/show
  - POST /api/copy
  - DELETE /api/delete
  - POST /api/pull
  - POST /api/push
  - POST /api/embed
  - GET /api/ps

---

# 1. /api/generate 接口参数概览

&emsp;&emsp;该接口使用提供的模型为给定提示生成响应。这是一个流式端点，因此会有一系列响应。最终响应对象将包括统计信息和请求中的其他数据。其中比较重要的参数我做了标红处理，大家重点理解。



<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>常规参数</font></p>
<div class="center">

| 参数名      | 类型      | 描述                                                         |
| ----------- | --------- | ------------------------------------------------------------ |
| <font color="red">**model**</font>   | *(必需)*  | 模型名称，必须遵循 `model:tag` 格式，如果不提供，则将默认为 `latest`。 |
| <font color="red">**prompt**</font>  | *(必需)*  | 用于生成响应的提示。                                         |
| **suffix**  | *(可选)*  | 模型响应后的文本。                                         |
| **images**  | *(可选)*  | base64 编码图像的列表（适用于多模态模型，如 llava）。      |

</div>


<p align="center"><font face="黑体" size=4> 高级参数 (可选)</font></p>
<div class="center">

| 参数名      | 类型      | 描述                                                         |
| ----------- | --------- | ------------------------------------------------------------ |
| <font color="red">**format**</font>  | *(可选)*  | 返回响应的格式。格式可以是 `json` 或 JSON 模式。<font color="red">最主要的问题是避免产生大量空格</font>         |
| <font color="red">**options**</font> | *(可选)*  | 文档中列出的其他模型参数，例如 `temperature`。              |
| <font color="red">**system**</font>  | *(可选)*  | 系统消息，用于覆盖 Modelfile 中定义的内容。                 |
| **template**| *(可选)*  | 要使用的提示模板，覆盖 Modelfile 中定义的内容。             |
| <font color="red">**stream**</font>  | *(可选)*  | 如果为 `false`，响应将作为单个响应对象返回，而不是对象流。 |
| **raw**     | *(可选)*  | 如果为 `true`，则不会对提示应用格式。                       |
| <font color="red">**keep_alive**</font> | *(可选)* | 控制模型在请求后保持加载的时间（默认：5分钟）。             |
| **context** | *(可选)*  | *(已弃用)* 从先前请求返回的上下文参数，用于保持简短的对话记忆。 |

</div>

&emsp;&emsp;其中，Options参数详细解释如下，同样我对重点参数做了标红处理，大家重点理解。

| 参数名 | 描述 | 值类型 | 示例用法 |
| --------------- | ------------------------------------------------------------ | ------ | ---------------------- |
| mirostat | 启用 Mirostat 采样以控制困惑度。（默认：0，0 = 禁用，1 = Mirostat，2 = Mirostat 2.0） | int | mirostat 0 |
| mirostat_eta| 影响算法对生成文本反馈的响应速度。较低的学习率会导致调整较慢，而较高的学习率会使算法更具响应性。（默认：0.1） | float | mirostat_eta 0.1 |
| mirostat_tau| 控制输出的连贯性和多样性之间的平衡。较低的值会导致更集中和连贯的文本。（默认：5.0） | float | mirostat_tau 5.0 |
| <font color="red">num_ctx</font> | 设置用于生成下一个标记的上下文窗口大小。（默认：2048）, 影响的是模型可以一次记住的最大 token 数量。 | int | num_ctx 4096|
| repeat_last_n| 设置模型回溯的范围以防止重复。（默认：64，0 = 禁用，-1 = num_ctx） | int | repeat_last_n 64 |
| repeat_penalty| 设置惩罚重复的强度。较高的值（例如 1.5）会更强烈地惩罚重复，而较低的值（例如 0.9）会更宽松。（默认：1.1） | float | repeat_penalty 1.1 |
| <font color="red">temperature</font> | 模型的温度。增加温度会使模型的回答更具创造性。（默认：0.8） | float | temperature 0.7 |
| seed | 设置用于生成的随机数种子。将其设置为特定数字将使模型对相同提示生成相同的文本。（默认：0） | int | seed 42 |
| <font color="red">stop</font> | 设置使用的停止序列。当遇到此模式时，LLM 将停止生成文本并返回。可以通过在 modelfile 中指定多个单独的停止参数来设置多个停止模式。 | string | stop "AI assistant:" |
| <font color="red">num_predict</font> | 生成文本时要预测的最大标记数。（默认：-1，无限生成）,影响模型最大可以生成的 token 数量。 | int | num_predict 42 |
| top_k | 降低生成无意义文本的概率。较高的值（例如 100）会给出更多样化的答案，而较低的值（例如 10）会更保守。（默认：40） | int | top_k 40 |
| top_p | 与 top-k 一起工作。较高的值（例如 0.95）会导致更具多样性的文本，而较低的值（例如 0.5）会生成更集中和保守的文本。（默认：0.9） | float | top_p 0.9 |
| min_p | top_p 的替代方案，旨在确保质量和多样性之间的平衡。参数 p 表示考虑标记的最小概率，相对于最可能标记的概率。例如，p=0.05 时，最可能的标记概率为 0.9，值小于 0.045 的 logits 会被过滤掉。（默认：0.0） | float | min_p 0.05 |


&emsp;&emsp;对于`endpoints`来说，如果使用代码调用，常规的调用方式是通`requests`库进行调用。如下所示：

In [2]:
import requests # type: ignore
import json

# 设置 API 端点
generate_url = "http://localhost:11434/api/generate"    # 这里需要根据实际情况进行修改

# 示例数据
generate_payload = {
    "model": "deepseek-r1:32b",   # 这里需要根据实际情况进行修改
    "prompt": "请生成一个关于人工智能的简短介绍。",  # 这里需要根据实际情况进行修改
    "stream": False,       # 默认使用的是True，如果设置为False，则返回的是一个完整的响应，而不是一个流式响应
}

# 调用生成接口
response_generate = requests.post(generate_url, json=generate_payload)
if response_generate.status_code == 200:
    generate_response = response_generate.json()
    print("生成响应:", json.dumps(generate_response, ensure_ascii=False, indent=2))
else:
    print("生成请求失败:", response_generate.status_code, response_generate.text)

生成响应: {
  "model": "deepseek-r1:32b",
  "created_at": "2025-10-03T14:39:14.149915367Z",
  "response": "<think>\n好的，用户让我生成一段关于人工智能的简短介绍。首先，我需要明确什么是人工智能的基本定义，确保涵盖主要概念。\n\n然后，得提到它的主要目标，比如模拟人类智能、学习和适应能力。接着，可以列举一些关键技术，如机器学习、深度学习和自然语言处理，这样内容更具体。\n\n还要说明AI的应用领域，比如医疗、金融、交通等，让读者了解其广泛性。最后，补充一下伦理和社会影响的问题，这样介绍会更全面。\n\n要保持简洁明了，用词通俗易懂，避免过于专业的术语，确保适合各种背景的读者。\n</think>\n\n人工智能（Artificial Intelligence, AI）是指模拟人类智能的系统或机器，能够执行如学习、推理和问题解决等任务。通过算法和技术，AI从数据中获取知识，并逐步改进性能。应用广泛于医疗、金融、交通等领域，正深刻改变我们的生活方式。同时，需关注伦理和社会影响，确保技术为社会福祉服务。",
  "done": true,
  "done_reason": "stop",
  "context": [
    151644,
    14880,
    43959,
    46944,
    101888,
    104455,
    9370,
    98237,
    99534,
    100157,
    1773,
    151645,
    151648,
    198,
    99692,
    3837,
    20002,
    104029,
    43959,
    104383,
    101888,
    104455,
    9370,
    98237,
    99534,
    100157,
    1773,
    101140,
    3837,
    35946,
    85106,
    100692,
    106582,
    104455,
    105166,
    91282,
    3837,
    103944,
    10

&emsp;&emsp;返回的响应中包含以下参数，其对应的描述如下：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>响应参数</font></p>
<div class="center">

| 参数名                  | 描述                                                         |
| ----------------------- | ------------------------------------------------------------ |
| **total_duration**      | 单次响应花费的总时间                                          |
| **load_duration**       | 加载模型花费的时间                                   |
| **prompt_eval_count**   | 提示中的token数                                               |
| **prompt_eval_duration**| 评估提示所花费的时间（以纳秒为单位）                                 |
| **eval_count**          | 响应中的token数                                               |
| **eval_duration**       | 生成响应的时间（以纳秒为单位）                              |
| **context**             | 在此响应中使用的对话的编码，可以在下一个请求中发送以保持对话记忆 |
| **response**            | 空响应是流的，如果未流式传输，则将包含完整的响应             |

</div>

&emsp;&emsp;返回的响应体中重点关注以下几个参数：

# 2. response 参数格式化解析

&emsp;&emsp;`response` 字段指的是模型生成的实际输出内容。对于 `DeepSeek-R1` 模型来说，`response` 字段中包含<think> 标签和正常文本，<think> 标签用于表示模型的思考过程或内部推理，而正常的文本则是模型生成的实际输出内容。注意：非推理类模型的返回结果中没有<think></think>标识。


In [3]:
generate_response["response"]

'<think>\n好的，用户让我生成一段关于人工智能的简短介绍。首先，我需要明确什么是人工智能的基本定义，确保涵盖主要概念。\n\n然后，得提到它的主要目标，比如模拟人类智能、学习和适应能力。接着，可以列举一些关键技术，如机器学习、深度学习和自然语言处理，这样内容更具体。\n\n还要说明AI的应用领域，比如医疗、金融、交通等，让读者了解其广泛性。最后，补充一下伦理和社会影响的问题，这样介绍会更全面。\n\n要保持简洁明了，用词通俗易懂，避免过于专业的术语，确保适合各种背景的读者。\n</think>\n\n人工智能（Artificial Intelligence, AI）是指模拟人类智能的系统或机器，能够执行如学习、推理和问题解决等任务。通过算法和技术，AI从数据中获取知识，并逐步改进性能。应用广泛于医疗、金融、交通等领域，正深刻改变我们的生活方式。同时，需关注伦理和社会影响，确保技术为社会福祉服务。'

&emsp;&emsp;可以通过简单的字符串操作来分离 <think> 标签中的思考内容和正常的文本内容，代码如下：

In [4]:
# 提取 <think> 标签中的内容
think_start = generate_response["response"].find("<think>")
think_end = generate_response["response"].find("</think>")

if think_start != -1 and think_end != -1:
    think_content = generate_response["response"][think_start + len("<think>"):think_end].strip()
else:
    think_content = "No think content found."

# 提取正常的文本内容
normal_content = generate_response["response"][think_end + len("</think>"):].strip()

# 打印结果
print("思考内容:\n", think_content)
print("\n正常内容:\n", normal_content)

思考内容:
 好的，用户让我生成一段关于人工智能的简短介绍。首先，我需要明确什么是人工智能的基本定义，确保涵盖主要概念。

然后，得提到它的主要目标，比如模拟人类智能、学习和适应能力。接着，可以列举一些关键技术，如机器学习、深度学习和自然语言处理，这样内容更具体。

还要说明AI的应用领域，比如医疗、金融、交通等，让读者了解其广泛性。最后，补充一下伦理和社会影响的问题，这样介绍会更全面。

要保持简洁明了，用词通俗易懂，避免过于专业的术语，确保适合各种背景的读者。

正常内容:
 人工智能（Artificial Intelligence, AI）是指模拟人类智能的系统或机器，能够执行如学习、推理和问题解决等任务。通过算法和技术，AI从数据中获取知识，并逐步改进性能。应用广泛于医疗、金融、交通等领域，正深刻改变我们的生活方式。同时，需关注伦理和社会影响，确保技术为社会福祉服务。


&emsp;&emsp;当然也可以用相同的方式提取返回的响应中所有参数的值：

In [5]:
# 打印每个参数的值
print("Model:", generate_response["model"])
print("Created At:", generate_response["created_at"])
print("Response:", generate_response["response"])
print("Done:", generate_response["done"])
print("Done Reason:", generate_response["done_reason"])
print("Context:", generate_response["context"])
print("Total Duration:", generate_response["total_duration"])
print("Load Duration:", generate_response["load_duration"])
print("Prompt Eval Count:", generate_response["prompt_eval_count"])
print("Prompt Eval Duration:", generate_response["prompt_eval_duration"])
print("Eval Count:", generate_response["eval_count"])
print("Eval Duration:", generate_response["eval_duration"])

Model: deepseek-r1:32b
Created At: 2025-10-03T14:39:14.149915367Z
Response: <think>
好的，用户让我生成一段关于人工智能的简短介绍。首先，我需要明确什么是人工智能的基本定义，确保涵盖主要概念。

然后，得提到它的主要目标，比如模拟人类智能、学习和适应能力。接着，可以列举一些关键技术，如机器学习、深度学习和自然语言处理，这样内容更具体。

还要说明AI的应用领域，比如医疗、金融、交通等，让读者了解其广泛性。最后，补充一下伦理和社会影响的问题，这样介绍会更全面。

要保持简洁明了，用词通俗易懂，避免过于专业的术语，确保适合各种背景的读者。
</think>

人工智能（Artificial Intelligence, AI）是指模拟人类智能的系统或机器，能够执行如学习、推理和问题解决等任务。通过算法和技术，AI从数据中获取知识，并逐步改进性能。应用广泛于医疗、金融、交通等领域，正深刻改变我们的生活方式。同时，需关注伦理和社会影响，确保技术为社会福祉服务。
Done: True
Done Reason: stop
Context: [151644, 14880, 43959, 46944, 101888, 104455, 9370, 98237, 99534, 100157, 1773, 151645, 151648, 198, 99692, 3837, 20002, 104029, 43959, 104383, 101888, 104455, 9370, 98237, 99534, 100157, 1773, 101140, 3837, 35946, 85106, 100692, 106582, 104455, 105166, 91282, 3837, 103944, 102994, 99558, 101290, 3407, 101889, 3837, 49828, 104496, 104121, 99558, 100160, 3837, 101912, 105717, 103971, 100168, 5373, 100134, 33108, 104117, 99788, 1773, 102524, 3837, 73670, 118569, 101883, 114876, 3837, 29

&emsp;&emsp;`Ollama` 返回的响应中，采用的时间单位均以纳秒返回。纳秒（nanosecond）和秒（second）之间的关系是：<font color="red">1 秒 = 10⁹ 纳秒</font>



In [6]:
# 将纳秒转换为秒
total_duration_s = generate_response["total_duration"] / 1_000_000_000
load_duration_s = generate_response["load_duration"] / 1_000_000_000
prompt_eval_duration_s = generate_response["prompt_eval_duration"] / 1_000_000_000
eval_duration_s = generate_response["eval_duration"] / 1_000_000_000

# 打印转换后的秒值
print("单次调用总花费时间:", total_duration_s)
print("加载模型花费时间:", load_duration_s)
print("评估提示所花费的时间:", prompt_eval_duration_s)
print("生成响应的时间:", eval_duration_s)

单次调用总花费时间: 6.496974502
加载模型花费时间: 0.228863547
评估提示所花费的时间: 0.025569951
生成响应的时间: 6.240912171


# 3. num_ctx / num_predict 输入输出控制

&emsp;&emsp;`num_ctx` 和 `num_predict`参数都是需要放置在 `options` 参数中的，其中：

- `num_ctx`该参数指的是大模型在一次对话中能够"看到"和"记住"的最大上下文长度，默认配置 2048，相当于一次只能向模型输入 2k `token`，超过 2k 模型就无法记住。当 `prompt` 特别长时往往会出现问题。并且现在开源模型往往支持长上下文，默认配置会严重限制本地模型能力。

- `num_predict` 参数指的是模型响应返回的最大 token 数据量。

&emsp;&emsp;我们可以这样测试：

In [7]:
import requests # type: ignore
import json

# 设置 API 端点
generate_url = "http://localhost:11434/api/generate"    # 这里需要根据实际情况进行修改

# 示例数据
generate_payload = {
    "model": "deepseek-r1:32b",   # 这里需要根据实际情况进行修改
    "prompt": "请生成一个关于人工智能的简短介绍。",  # 这里需要根据实际情况进行修改
    "stream": False,       # 默认使用的是True，如果设置为False，则返回的是一个完整的响应，而不是一个流式响应
    "options": {
        # "num_ctx": 7,  慎用，可能会导致Ollama服务不稳定，建议选择 1024 及以上
        "num_predict": 10
    }
}

# 调用生成接口
response_generate = requests.post(generate_url, json=generate_payload)
if response_generate.status_code == 200:
    generate_response = response_generate.json()
    print("生成响应:", json.dumps(generate_response, ensure_ascii=False, indent=2))
else:
    print("生成请求失败:", response_generate.status_code, response_generate.text)

生成响应: {
  "model": "deepseek-r1:32b",
  "created_at": "2025-10-03T14:42:27.181479878Z",
  "response": "<think>\n嗯，用户让我生成一个关于人工智能",
  "done": true,
  "done_reason": "length",
  "context": [
    151644,
    14880,
    43959,
    46944,
    101888,
    104455,
    9370,
    98237,
    99534,
    100157,
    1773,
    151645,
    151648,
    198,
    106287,
    3837,
    20002,
    104029,
    43959,
    46944,
    101888,
    104455
  ],
  "total_duration": 537172829,
  "load_duration": 218187733,
  "prompt_eval_count": 13,
  "prompt_eval_duration": 25298913,
  "eval_count": 10,
  "eval_duration": 292216403
}


# 4. 流式输出功能

&emsp;&emsp;接下来看流式输出输出，其参数和如上代码保持一致，只需要在 `response_generate` 中添加 `stream=True`，最后再通过流式的方式进行响应结果处理即可。代码如下所示：

> 这里有一个使用`DeepSeek-R1`的小技巧，将温度即`temperature`设置在0.5-0.7（建议0.6）的范围内，可以有效防止无尽的重复或不连贯的输出。

In [11]:
import requests # type: ignore
import json

# 设置 API 端点
generate_url = "http://localhost:11434/api/generate"

# 示例数据
generate_payload = {
    "model": "deepseek-r1:32b",
    "prompt": "请生成一个关于人工智能的简短介绍。",
    "options": {
        "temperature": 0.6, 
    }
}

# 调用生成接口
response_generate = requests.post(generate_url, json=generate_payload, stream=True)  # 在这里添加stream=True
if response_generate.status_code == 200:
    # 处理流式响应
    for line in response_generate.iter_lines(): 
        if line:
            try:
                # 解码并解析每一行的 JSON
                response_json = json.loads(line.decode('utf-8'))
                if 'response' in response_json:
                    print(response_json['response'], end='', flush=True)

                # 检查 response_json 字典中是否存在键 'done'，并且其值是否为 True。如果这个条件成立，表示生成的响应已经完成。
                if response_json.get('done', False):
                    print('\n\n完整响应:', json.dumps(response_json, ensure_ascii=False, indent=2))
            except json.JSONDecodeError as e:
                print(f"JSON 解析错误: {e}")
else:
    print("生成请求失败:", response_generate.status_code, response_generate.text)

<think>
嗯，用户让我写一个关于人工智能的简短介绍。首先，我得考虑用户是谁，可能是学生、老师还是普通对科技感兴趣的人。他们可能需要一个简洁明了的概述，不需要太深入的技术细节。

接下来，我应该涵盖人工智能的基本定义，比如它是什么，有什么核心领域，比如机器学习和深度学习。还要提到它的应用范围，比如医疗、金融、交通这些领域，这样用户能理解AI的实际影响。

另外，伦理和社会问题也是重要的一部分，不能只讲优点，也要提挑战，比如隐私和就业问题。最后，展望未来，强调技术进步带来的可能性，但也要注意平衡发展。

结构上，我需要先定义什么是人工智能，然后介绍它的核心技术，接着举例应用，再讨论挑战，最后总结未来的潜力。这样逻辑清晰，用户容易理解。
</think>

人工智能（Artificial Intelligence, AI）是指模拟人类智能的系统或机器，能够执行如学习、推理、问题解决和自然语言处理等任务。它通过算法和大数据分析，使计算机具备类似人类的决策和适应能力。AI的核心技术包括机器学习、深度学习和自然语言处理等。广泛应用于医疗、金融、交通等领域，为社会带来效率提升和创新解决方案，但也引发隐私、伦理和社会影响等方面的讨论。

完整响应: {
  "model": "deepseek-r1:32b",
  "created_at": "2025-10-03T15:06:41.749071697Z",
  "response": "",
  "done": true,
  "done_reason": "stop",
  "context": [
    151644,
    14880,
    43959,
    46944,
    101888,
    104455,
    9370,
    98237,
    99534,
    100157,
    1773,
    151645,
    151648,
    198,
    106287,
    3837,
    20002,
    104029,
    61443,
    46944,
    101888,
    104455,
    9370,
    98237,
    99534,
    100157,
    1773,
    101140,
   

下面提供一个优化版本

In [12]:
import requests  # type: ignore
import json

# 设置 API 端点
generate_url = "http://localhost:11434/api/generate"

# 示例数据
# 注意: 只有 DeepSeek-R1 系列模型会输出思考过程 <think>...</think>
generate_payload = {
    "model": "deepseek-r1:32b",  # 使用 DeepSeek-R1 才能看到思考过程
    "prompt": "请详细解释一下快速排序算法的原理和时间复杂度。",
    "stream": True,
    "options": {
        "temperature": 0.6,
        #"num_predict": 1000  # DeepSeek-R1 思考过程较长，增加输出限制
    }
}

print("=" * 60)
print("开始生成内容...")
print("=" * 60)
print()

# 状态跟踪
in_thinking = False
last_content = ""  # 用于检测重复内容

try:
    with requests.post(generate_url, json=generate_payload, stream=True, timeout=120) as response:
        if response.status_code == 200:
            for line in response.iter_lines():
                if line:
                    try:
                        data = json.loads(line)
                        
                        if "response" in data:
                            content = data["response"]
                            
                            # 跳过空内容
                            if not content or content.isspace():
                                continue
                            
                            # 检测思考过程标记
                            if "<think>" in content or "<reasoning>" in content:
                                if not in_thinking:
                                    print("\n【💭 思考过程】")
                                    print("-" * 60)
                                    in_thinking = True
                                content = content.replace("<think>", "").replace("<reasoning>", "")
                            
                            if "</think>" in content or "</reasoning>" in content:
                                content = content.replace("</think>", "").replace("</reasoning>", "")
                                if in_thinking:  # 只在真的有思考过程时才打印分隔
                                    print("\n" + "-" * 60)
                                    print("\n【✨ 最终答案】")
                                    print("-" * 60)
                                in_thinking = False
                            
                            # 输出内容（流式输出不应该有重复）
                            print(content, end='', flush=True)
                            last_content = content
                        
                        # 完成时打印统计
                        if data.get("done", False):
                            print("\n\n" + "=" * 60)
                            print("✅ 生成完成")
                            print("=" * 60)
                            
                            # 统计信息
                            if "total_duration" in data:
                                print(f"总耗时: {data['total_duration'] / 1e9:.2f} 秒")
                            if "eval_count" in data and "eval_duration" in data:
                                tokens = data["eval_count"]
                                duration = data["eval_duration"] / 1e9
                                speed = tokens / duration if duration > 0 else 0
                                print(f"生成 tokens: {tokens} ({speed:.2f} tokens/秒)")
                            if "prompt_eval_count" in data:
                                print(f"输入 tokens: {data['prompt_eval_count']}")
                            break
                            
                    except json.JSONDecodeError as e:
                        print(f"\n解析错误: {e}")
                        continue
        else:
            print(f"❌ 请求失败: {response.status_code}")
            print(f"错误: {response.text}")
            
except requests.exceptions.RequestException as e:
    print(f"❌ 请求异常: {e}")
    print("提示: 请确保 Ollama 服务正在运行")



开始生成内容...


【💭 思考过程】
------------------------------------------------------------
嗯，我现在要仔细理解一下快速排序算法的原理以及它的时间复杂度。我对快速排序有一些基本了解，但还不够深入，所以需要一步一步地梳理。

首先，快速排序是一种高效的排序算法，属于分治法的一种。我记得分治法的基本思想是将问题分解成更小的部分来解决，然后再合并结果。那快速排序是怎么应用这个方法的呢？

快速排序的大致步骤应该是这样的：选择一个基准元素（pivot），然后将数组分成两部分，一部分小于等于基准元素，另一部分大于等于基准元素。接着递归地对这两部分进行同样的操作，直到整个数组有序。

那么，关键点在于如何选择基准元素和分区过程。我之前学过几种选择基准的方法，比如选第一个元素、最后一个元素，或者中间的某个位置，甚至随机选择。不同的选择会影响算法的性能，特别是最坏情况下时间复杂度的表现。

接下来是分区的过程，这一步应该是快速排序的核心部分。假设我们已经选择了基准元素，那么我们需要将数组分成两部分：一部分所有元素都小于等于基准，另一部分则大于等于基准。具体来说，通常会用到双指针的方法，一个从左往右找比基准大的元素，另一个从右往左找比基准小的元素，然后交换它们的位置。这样，基准元素最终会被放在正确的位置上。

举个例子可能会有帮助。比如数组是 [3,6,8,10,1,4,7]，假设我们选择第一个元素3作为基准。那么在分区过程中，左边的指针从位置0开始，找到第一个比3大的数，也就是6；右边的指针从末尾开始，找到第一个比3小的数，比如1。然后交换这两个数的位置，数组变成 [1,6,8,10,3,4,7]。接着继续这个过程，直到左右指针相遇，这样基准元素就被正确地放置在了中间位置。

接下来，递归地对左边的子数组[1]和右边的子数组[6,8,10,3,4,7]进行同样的操作。对于右边的子数组，可能选择第一个元素6作为基准，然后分区后得到 [3,4,6,8,10,7]，然后再继续递归。

关于时间复杂度，快速排序在平均情况下是O(n log n)，这比冒泡排序之类的O(n²)要好得多。但是最坏情况下，比如每次选择的基准都是数组中最小或最大的元素，这样分区后的子数组长度分别为n-1和0，导致递归深度达到n层，每层操作需要O

两个代码的详细对比

| 特性 | fufan 的代码 | 我写的代码 |
|------|------------|-----------|
| **核心功能** | ✅ 基础流式输出 | ✅ 增强流式输出 |
| **思考过程识别** | ❌ 无 | ✅ 自动识别 `<think>` 标签 |
| **输出格式化** | ❌ 无 | ✅ 区分思考过程和答案 |
| **统计信息** | ❌ 只打印完整JSON | ✅ 格式化统计（速度、耗时等）|
| **异常处理** | ⚠️ 基础 | ✅ 完善（网络、超时等）|
| **资源管理** | ⚠️ 手动管理 | ✅ 使用 `with` 自动管理 |
| **空白过滤** | ❌ 无 | ✅ 跳过空白内容 |
| **超时设置** | ❌ 无 | ✅ 120秒超时 |
| **视觉效果** | ⚠️ 简单 | ✅ 分隔线、标题等 |

# 5. Ollama 模型生命周期管理

&emsp;&emsp;默认情况下，<font color="red">**通过`Ollama run`启动一个模型后，会将其在VRAM(显存)中保存5分钟**</font>。主要作用是为了做性能优化，通过保持模型在显存中，可以避免频繁的加载和卸载操作，从而提高响应速度，特别是在连续请求的情况下。

- <font color="red">**keep_alive**</font>

&emsp;&emsp;我们可以通过`ollama stop` 命令立即卸载某个模型。而在生成请求中，一种高效的方式是通过`keep_alive`参数来控制模型在请求完成后保持加载在内存中的时间。其可传入的参数规则如下：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>keep_alive 参数类型</font></p>
<div class="center">

| 参数类型               | 示例         | 描述                                       |
|------------------------|--------------|--------------------------------------------|
| 持续时间字符串         | "10m" 或 "24h" | 表示保持模型在内存中的时间，单位可以是分钟（m）或小时（h）。 |
| 以秒为单位的数字       | 3600         | 表示保持模型在内存中的时间，单位为秒。   |
| 任何负数               | -1 或 "-1m"  | 表示保持模型在内存中，负数值将使模型持续加载。 |
| '0'                    | 0            | 表示在生成响应后立即卸载模型。             |

</div>


In [9]:
import requests # type: ignore
import json

# 设置 API 端点
generate_url = "http://localhost:11434/api/generate"

# 示例数据
generate_payload = {
    "model": "deepseek-r1:32b",
    "prompt": "请生成一个关于人工智能的简短介绍。",
    "stream": False,
    "keep_alive": "10m",   # 设置模型在请求后保持加载的时间
    "options": {
        "temperature": 0.6,
    }
}

# 调用生成接口
response_generate = requests.post(generate_url, json=generate_payload)
if response_generate.status_code == 200:
    generate_response = response_generate.json()
    print("生成响应:", json.dumps(generate_response, ensure_ascii=False, indent=2))
else:
    print("生成请求失败:", response_generate.status_code, response_generate.text)



if generate_response["eval_duration"] != 0:
    tokens_per_second = generate_response["eval_count"] / generate_response["eval_duration"] * 10**9
    print(f"Tokens per second: {tokens_per_second}")
else:
    print("eval_duration is zero, cannot calculate tokens per second.")

生成响应: {
  "model": "deepseek-r1:32b",
  "created_at": "2025-10-03T14:45:59.43996011Z",
  "response": "<think>\n嗯，用户让我生成一个关于人工智能的简短介绍。首先，我得想想什么是AI的核心概念，可能包括定义、应用领域和一些关键的技术点。\n\n然后，用户可能对AI的应用感兴趣，比如机器学习、自然语言处理这些方面。我应该涵盖这些内容，同时保持简洁明了。\n\n还要注意不要太技术化，让读者容易理解。或许可以提到一些实际的例子，比如智能助手或自动驾驶，这样更生动。\n\n最后，总结一下AI的影响和未来发展的可能性，让用户有个全面的认识。\n</think>\n\n人工智能（Artificial Intelligence, AI）是指由人创造的能够执行通常需要人类智能的任务的系统或机器。这些任务包括学习、推理、问题解决、感知和语言理解等。AI技术广泛应用于多个领域，如自动驾驶、语音助手、医疗诊断、金融分析和娱乐推荐等，极大地改变了我们的生活方式和工作方式。随着技术的进步，人工智能正变得越来越智能化和自主化，为未来带来了无限可能。",
  "done": true,
  "done_reason": "stop",
  "context": [
    151644,
    14880,
    43959,
    46944,
    101888,
    104455,
    9370,
    98237,
    99534,
    100157,
    1773,
    151645,
    151648,
    198,
    106287,
    3837,
    20002,
    104029,
    43959,
    46944,
    101888,
    104455,
    9370,
    98237,
    99534,
    100157,
    1773,
    101140,
    3837,
    35946,
    49828,
    105839,
    106582,
    15469,
    104867,
    101290,
    3837,
    87

&emsp;&emsp;此时就可以在服务器控制台查看到，`deepseek-r1:32b`模型将可以在显存中保持10分钟。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502131907776.png" width=100%></div>

&emsp;&emsp;`keep_alive` 在工程化的项目中，往往需要根据请求的频率来设置，如果请求不频繁，可以使用默认值或较短的时间，以便在不使用时释放内存。而如果应用程序需要频繁调用模型，可以设置较长的 `keep_alive` 时间，以减少加载时间。很关键，非常影响服务器的性能和应用程序的用户体验。大家一定要注意。

&emsp;&emsp;接下来我们进入下一个课件中了解 `/api/chat` 接口。
